In [1]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio
import json
from pathlib import Path
import yaml

In [ ]:
class TopicEvent:
    def __init__(self, machine: AsyncMachine, name: str, count: int = 0):
        self.machine = machine
        self.name = name
        self.count = count

    async def msg_callback(self, msg: str):
        print(f"Message received: {msg}")
        if msg == "gate":
            self.gate_count += 1
            if self.gate_count == 5:
                # Trigger the state machine "found" to make a transition to end
                self.gate_count = 0
                await self.machine.trigger("found")
        else:
            self.gate_count = 0

In [2]:
class Mission:
    def __init__(self, missions_pkg: str = "stingray_missions"):
        config_path = Path("src/stingray_missions/configs/mission.yaml")
        with open(config_path, "r") as f:
            self.config = yaml.safe_load(f)

        self.machine = AsyncMachine(
            model=self,
            states=["start", "finish"],
            transitions=[["abort", "*", "finish"]],
            initial="start",
            auto_transitions=False
        )
        self.machine.add_states(self.config["states"])
        self.machine.add_transitions(self.config["transitions"])

        if self.config["events"]:
            self.events = []
            for event in self.config["events"]:
                if event["type"] == "topic":
                    self.events.append(TopicEvent(self.machine, event["name"], event["count"]))

    

In [5]:
async def msg_loop(mission_obj: Mission):
    while True:
        await mission_obj.msg_callback("gate")
        await asyncio.sleep(1)

def main():
    mission = Mission()
    future = asyncio.wait([msg_loop(mission), mission.go()])
    asyncio.get_event_loop().run_until_complete(future)

In [6]:
main()

RuntimeError: This event loop is already running

In [6]:
mission.machine.get_transitions()

[<AsyncTransition('start', 'start')@139791018338784>,
 <AsyncTransition('finish', 'start')@139790991813904>,
 <AsyncTransition('rotate_clockwise', 'start')@139790991813520>,
 <AsyncTransition('rotate_counter_clockwise', 'start')@139790991814576>,
 <AsyncTransition('start', 'finish')@139790991813808>,
 <AsyncTransition('finish', 'finish')@139790991813856>,
 <AsyncTransition('rotate_clockwise', 'finish')@139790991813712>,
 <AsyncTransition('rotate_counter_clockwise', 'finish')@139790991814480>,
 <AsyncTransition('start', 'end')@139790991814096>,
 <AsyncTransition('finish', 'end')@139790991814144>,
 <AsyncTransition('start', 'rotate_clockwise')@139790991814672>,
 <AsyncTransition('finish', 'rotate_clockwise')@139790991813760>,
 <AsyncTransition('rotate_clockwise', 'rotate_clockwise')@139790991813472>,
 <AsyncTransition('rotate_counter_clockwise', 'rotate_clockwise')@139790991814384>,
 <AsyncTransition('start', 'rotate_counter_clockwise')@139790991814528>,
 <AsyncTransition('finish', 'rota